# Homework 4: Sentence Segmentation
Spencer Hann  
CS 562 | Winter 2019

Sentence segmentation is a crucial, yet under-appreciated, part of any text processing pipeline. In any real-world setting, text will not come to you pre-chunked into sentences, and if your pipeline involves any sentence-level analysis (parsing, translation, entity & co-reference extraction, etc.) the first thing you'll need to do will be split things up into sentences. This is a deceptively challenging problem, as punctuation can be used in multiple ways. Consider the following sentence (borrowed from Kyle Gorman's [excellent discussion of the subject](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/)): 

> _"Rolls-Royce Motor Cars Inc. said it expects its U.S. sales to remain steady at about 1,200 cars in 1990."_ 

This sentence features several periods, but only one of them (the last one) represents a valid sentence break. Sentences can also have embedded clauses or quotation marks, some of which may superficially look like a sentence boundary: 

> _He says the big questions–“Do you really need this much money to put up these investments? Have you told investors what is happening in your sector? What about your track record?–“aren’t asked of companies coming to market._

There are many ways to approach this particular problem; it can be approached using rules, which themselves may be encoded as regular expressions, or (more usefully) it can be modeled as a machine learning problem. Specifically, as a binary classification problem, where candidate sentence boundaries (i.e., punctuation marks from the closed class of English punctuation that can represent a sentence boundary) are either _positive_ examples (actual sentence boundaries) or _false_ (not sentence boundaries). 

In this assignment, you will develop and test your own sentence boundary detection algorithm. This assignment is shorter than our previous assignments, and is also much more open-ended. By this point in the class, you have been exposed to several different families of technique for text classification and analysis, and have hopefully begun to build up some intuitions. For this assignment, you can choose whatever methods you like: log-linear models, rule-based methods, some sort of neural network, it's up to you!

## 0. Setup & data exploration

In [1]:
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [2]:
from hw4_utils import data
from hw4_utils import classifier

We've provided a version of the WSJ section of the Penn Treebank to use for training and evaluation purposes. See `data/wsj_sbd/README.md` for a description of where this data came from and how it was assembled.

The `hw4_utils.data` module has a useful method for reading in data (`file_as_string`) and another utility function to extract candidate boundaries. Let's use these to explore our data a little bit.

In [3]:
dev = data.file_as_string("data/wsj_sbd/dev.txt")
train = data.file_as_string("data/wsj_sbd/train.txt")
test = data.file_as_string("data/wsj_sbd/test.txt")

In [4]:
dev[:100]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is c'

Each of these is a string containing the text, with one input sentence per line. Note that newline characters are still present, which will be important as this will provide us with our ground truth.

Remember, we _only_ will use the `test` split for our final evaluation. For data exploration, feature engineering, and for tuning hyperparameters, use the `dev` split.

The first and most important thing to find out when doing classification is what your class probabilities are, so let's look into that. The `load_candidates()` function in the `hw4_utils.data` module will read an input data split, identify candidate boundaries, and extract information about the candidates context that we may want to use for feature engineering. One of the attributes it will extract is whether or not the candidate is actually a true sentence break (`is_true_break`). Obviously, this would only be useful at training time, as in the "real world" we wouldn't know the answer. :-)

We will use `load_candidates()` to answer our question about class balance:

In [5]:
from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

As is often the case, our classes are quite imbalanced! Let's load in our ground truth labels:

In [6]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]

What else can we find out about our candidate breaks?

In [7]:
candidates = list(data.load_candidates(dev[:10000])) # for now, just get candidates from the first 10k characters
first_can = candidates[0]

In [8]:
print("original context: ",first_can.orig_obs)
print("punctuation mark: ", first_can.punctuation_mark)
print("token to the left: ", first_can.left_token)
print("token to the right: ", first_can.right_token)

original context:  d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El
punctuation mark:  .
token to the left:  Nov
token to the right:  29


Take a look at the the `Observation` namedtuple in `hw4_utils.data` for more details about the various attributes that you can find out about each possible sentence boundary.

## 1. Baselines

We will begin by looking at three different baselines. First, we will 

### Baseline #1: Majority class

Since our data are quite imbalanced, what if we always treat every candidate boundary as though it was a true boundary?

In [9]:
y_hat_baseline = [classifier.baseline_classifier(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_baseline))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1766
        True       0.77      1.00      0.87      5769

   micro avg       0.77      0.77      0.77      7535
   macro avg       0.38      0.50      0.43      7535
weighted avg       0.59      0.77      0.66      7535



/home/spencer/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Baseline #2: Next-token capitalization

What if we say that a candidate is a boundary if the following token is capitalized?

In [10]:
y_hat_next_tok_cap = [classifier.next_tok_capitalized_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_next_tok_cap))

              precision    recall  f1-score   support

       False       0.60      0.40      0.48      1766
        True       0.83      0.92      0.87      5769

   micro avg       0.80      0.80      0.80      7535
   macro avg       0.71      0.66      0.67      7535
weighted avg       0.78      0.80      0.78      7535



### Baseline #3: Punkt

Using the [NLTK implementation](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) of [Punkt (Kiss & Strunk, 2006)](https://www.mitpressjournals.org/doi/pdf/10.1162/coli.2006.32.4.485):

(Note: You'll need to have downloaded the NLTK pre-trained Punkt model for this baseline to work)

In [11]:
y_hat_punkt = [classifier.punkt_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_punkt))

              precision    recall  f1-score   support

       False       0.92      0.43      0.59      1766
        True       0.85      0.99      0.91      5769

   micro avg       0.86      0.86      0.86      7535
   macro avg       0.88      0.71      0.75      7535
weighted avg       0.87      0.86      0.84      7535



### Conclusion

None of these are especially inspiring in their performance... can you do better?

## 2. Your turn!

### Part 1: Now it's your turn.

Your mission, in this assignment, is to implement at least two additional approaches. The approaches must be different in terms of the features you choose to use, classification approach, or both, but beyond that limitation you are free to do whatever you like as long as it is your own work (i.e., do not download and use an already-existing sentence boundary detector tool). You may, however, replicate an existing algorithm, though please be very careful not about re-using other people's code. 

For a discussion of features that may prove useful, and of other approaches to this task that you may wish to replicate or build off of, consult the bibliography discussed in [Gorman (2014)](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/); note that the specific sentence boundary detector described on that page uses a small set of highly useful features, but you will want to go beyond simply replicating his system.

***Deliverable***: The code for your ≥ two additional classifiers, along with a brief writeup (≈1 page) of what you built and why you chose those specific approaches and features. 

### Part 2: Evaluation & Error analysis

After building and describing your additional classifiers, evaluate their performance against the `test` partition of the data as shown above. Write a brief summary of their relative performance, and include an _error analysis_. Examine cases where your classifiers failed to predict correctly, or disagreed with one another, and see if you can identify any patterns that might explain where they were going awry.

***Deliverable***: Your written summary, as well as any relevant data tables.

## Classifier 1: _Regex All the Things!!!_ (rule-based regex classifier)

In [12]:
import re
import numpy as np

In [13]:
train = list(data.load_candidates(data.file_as_string("data/wsj_sbd/train.txt")))
dev = list(data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt")))
test = list(data.load_candidates(data.file_as_string("data/wsj_sbd/test.txt")))

For my first classifier, I was adhering to the idea that simpler is better. Machine Learning methodologies can be incredibly powerful, but sometimes are inferior when simpler solution exists. This is because non-Machine Learning approaches can be more tractable, interpretable, and, sometimes, easier to implement/maintain. Also, sometimes it's more fun to come up with features yourself. Since sentence boundary detection seems like a simple task at face value (for humans, at least), I wanted to try a simple solution. 

I decided that an easy approach to this problem to use regexes to evaluate the contexts of end-sentence punctuation. For example, if the next word is capitalized or not is a decent indicator for sentence beginnings. First I created a common abbreviations list, `common_abbr`. This alone, actually, yielded surprising results, with results in the mid-90s. I had expected that my regex approach could get good results with respect to its simplicity, but since I had gotten such good results in my first version I decided to continue honing this rule-based method to see how far I could push it. (Note: in the first iteration `common_abbr` included titles, like "Ms" and "Prof", though they were moved to a dedicated list in a future iteration.)  

To improve the model I adjusted the rules (ways the regex matching was being interpreted) and I added as many more abbreviations/titles as I could think of. I looked at the examples that were classified incorrectly, searching not for specific instances of failure, but for errors that I imagined were very general. For example, my list of abbreviations did not include month abbreviations.

In [14]:
months = ['Jan', 'Feb', 'Mar', 'Apr','Jun','Jul','Aug','Sept','Sep','Oct','Nov','Dec']
months += [s.lower() for s in months]
months += [s.upper() for s in months]
months += [month+"(\.)*\s*[0-9]*" for month in months]
misc = ['St','Pl','Ave','Rd','Ct','Blvd','Inc','Co','Ltd','No','etc','i','e','g','et','al','Dept','Corp']
misc += [s.lower() for s in misc]

common_abbr = months + misc + ["abbr"] # lol, almost forgot to add "abbr" to common_abbr :/
common_abbr = [".*"+s for s in common_abbr]

titles = ['Dr','Mr','Mrs','Ms','Messrs','Prof','Dir','Exec','Asst','Sen','Gov',
          'Rep','Reps','Repr','Dep','St','Rev','Capt','Sgt','Lt']

common_abbr = re.compile('|'.join(common_abbr))
titles = re.compile('|'.join(titles))

initials = re.compile('([a-zA-Z]\.)*[a-zA-Z]')
lowercase = re.compile("([a-z]|[0-9]).*")
    
    
def regex_classifier(candidate, test_output=False):
    if common_abbr.match(candidate.left_raw) and lowercase.match(candidate.right_token) \
            or titles.fullmatch(candidate.left_token) \
            or initials.fullmatch(candidate.left_token): # not a true break
        if test_output and candidate.is_true_break: # I was wrong
            print("\nFalse Negative")
            print(candidate.left_token,', ', candidate.orig_obs)
        return False
    
    else: # is a true break
        if test_output and not candidate.is_true_break:
            print("\nFalse Positive")
            print(candidate.left_token,', ', candidate.orig_obs)
        return True

After honing this rule-based approach I was able to get my overall accuracy up to 99%, which I was happy with, especially considering this regex approach was originally meant as a precursor to a more complex Machine Learning approach.

In [15]:
y_train = [o.is_true_break for o in train]
results_train = [regex_classifier(o) for o in train]
print(classification_report(y_train, results_train))

y_dev = [o.is_true_break for o in dev]
results_dev = [regex_classifier(o) for o in dev]
print(classification_report(y_dev, results_dev))

y_test = [o.is_true_break for o in test]
results_test = [regex_classifier(o) for o in test]
print(classification_report(y_test, results_test))

              precision    recall  f1-score   support

       False       0.97      0.97      0.97      9453
        True       0.99      0.99      0.99     34826

   micro avg       0.99      0.99      0.99     44279
   macro avg       0.98      0.98      0.98     44279
weighted avg       0.99      0.99      0.99     44279

              precision    recall  f1-score   support

       False       0.97      0.99      0.98      1766
        True       1.00      0.99      0.99      5769

   micro avg       0.99      0.99      0.99      7535
   macro avg       0.98      0.99      0.99      7535
weighted avg       0.99      0.99      0.99      7535

              precision    recall  f1-score   support

       False       0.98      0.98      0.98      2205
        True       0.99      0.99      0.99      7531

   micro avg       0.99      0.99      0.99      9736
   macro avg       0.99      0.99      0.99      9736
weighted avg       0.99      0.99      0.99      9736



## Classifier 2: _Feed the Regex to a Support Vector Machine!!!_

In [16]:
from sklearn import svm

Well, the regex approach worked even better than I thought it would. So well, in fact, that I ended up putting in more work/spending more time on that than I originally intended. I also improved it more that I original intended/thought I could. So, I decided that an interesting second approach would be to take these regex features that served me so well in my first attempt and give them to a Support Vector Machine, to see if an SVM could learn better rules (coefficients) than I was able to manufacture manually.

In [17]:
n_features = 4

def gen_feature_vector(candidate, fvec):
    fvec[0] = +1 if common_abbr.match(candidate.left_raw)    else -1
    fvec[1] = +1 if lowercase.match(candidate.right_token)   else -1
    fvec[2] = +1 if titles.fullmatch(candidate.left_token)   else -1
    fvec[3] = +1 if initials.fullmatch(candidate.left_token) else -1
#     fvec[4] = +1 if lowercase.match(candidate.left_token)    else -1
    
def gen_all_feature_vectors(data):
    feature_vectors = np.empty((len(data),n_features))

    for item,vector in zip(data,feature_vectors):
        gen_feature_vector(item, vector)
    
    return feature_vectors, np.asarray([o.is_true_break for o in data])

def classifier_test(classifier, x, y):
    y_hat = classifier.predict(x)
    print(classification_report(y_hat, y))

In [18]:
x_train, y_train = gen_all_feature_vectors(train)
x_dev,   y_dev   = gen_all_feature_vectors(dev)
x_test,  y_test  = gen_all_feature_vectors(test)

In [19]:
classifier = svm.SVC(kernel="linear")
classifier.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [20]:
classifier_test(classifier, x_train, y_train)
classifier_test(classifier, x_dev, y_dev)
classifier_test(classifier, x_test, y_test)

              precision    recall  f1-score   support

       False       0.97      0.97      0.97      9473
        True       0.99      0.99      0.99     34806

   micro avg       0.99      0.99      0.99     44279
   macro avg       0.98      0.98      0.98     44279
weighted avg       0.99      0.99      0.99     44279

              precision    recall  f1-score   support

       False       0.99      0.97      0.98      1800
        True       0.99      1.00      0.99      5735

   micro avg       0.99      0.99      0.99      7535
   macro avg       0.99      0.98      0.99      7535
weighted avg       0.99      0.99      0.99      7535

              precision    recall  f1-score   support

       False       0.98      0.98      0.98      2194
        True       0.99      0.99      0.99      7542

   micro avg       0.99      0.99      0.99      9736
   macro avg       0.99      0.99      0.99      9736
weighted avg       0.99      0.99      0.99      9736



These results are almost identical the results from my regex model. In fact, they are exactly identical, except that the "precision" and "recall" columns are switched, but only on the dev set (those columns are identical for the train and test sets so "switching" doesn't really mean anything there). Even though the columns are switched, though, they still contain the exact same values that they did for the regex classifier. It is possible that the SVM is prioritizing recall over precision somehow.

### Feature Reduction

When I first created the features for the SVM, I added a feature for the `lowercase` regex matching the right token, as well as the left, even though this was not considered in my regex classifier. It did not have any impact on the SVM though, so I commented it out.  
I decided, then, to go through all the other features, removing them individually, to see how they affected the SVM's ability to learn. All the remaining features had significant impacts, greatly reducing the SVM accuracies when removed, except for the `common_abbr` regex. This surprised me, since it was such an important part of my first classifier, so I decided to include these results.  
I also experimented with different kernels for the SVMs, with full and reduced features sets, but did not include those tests since the results were the same, regardless of kernel.

In [21]:
# Remove first feature (common_abbr) from all feature vectors
x_train = x_train[:,1:]
x_dev = x_dev[:,1:]
x_test = x_test[:,1:]

n_features -= 1

In [22]:
classifier = svm.SVC(kernel="linear")
classifier.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

### Performance on Reduced Feature Set

In [23]:
classifier_test(classifier, x_train, y_train)
classifier_test(classifier, x_dev, y_dev)
classifier_test(classifier, x_test, y_test)

              precision    recall  f1-score   support

       False       0.97      0.97      0.97      9473
        True       0.99      0.99      0.99     34806

   micro avg       0.99      0.99      0.99     44279
   macro avg       0.98      0.98      0.98     44279
weighted avg       0.99      0.99      0.99     44279

              precision    recall  f1-score   support

       False       0.99      0.97      0.98      1800
        True       0.99      1.00      0.99      5735

   micro avg       0.99      0.99      0.99      7535
   macro avg       0.99      0.98      0.99      7535
weighted avg       0.99      0.99      0.99      7535

              precision    recall  f1-score   support

       False       0.98      0.98      0.98      2194
        True       0.99      0.99      0.99      7542

   micro avg       0.99      0.99      0.99      9736
   macro avg       0.99      0.99      0.99      9736
weighted avg       0.99      0.99      0.99      9736

